In [28]:


def file_as_bytes(file):
    with file:
        return file.read()
def file_md5sum(file):
    import hashlib
    try:
        return hashlib.md5(file_as_bytes(open(file, 'rb'))).hexdigest()
    except:
        print ("md5sum error", file)
        return False
    
def push_panda_table_sql(connection, table_name, df):
    
    mycursor=connection.cursor()
    mycursor.execute("SHOW TABLES LIKE '"+table_name+"'")
    result = mycursor.fetchone()
    if not result:
        cols = "`,`".join([str(i) for i in df.columns.tolist()])
        db_to_crete = "CREATE TABLE `"+table_name+"` ("+' '.join(["`"+x+"` REAL," for x in df.columns.tolist()])[:-1]+")"
        print ("[Table {:s} created into SQL Server]".format(table_name))
        mycursor = connection.cursor()
        mycursor.execute(db_to_crete)

    cols = "`,`".join([str(i) for i in df.columns.tolist()])

    for i,row in df.iterrows():
        sql = "INSERT INTO `"+table_name+"` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        mycursor.execute(sql, tuple(row.astype(str)))
        connection.commit()

    mycursor.close()

    

def read_sql_value(connection, table_name, row_element, row_element_condition, 
                     colum_element, verbose=False):
    sql = "SELECT `"+colum_element+"` FROM `"+table_name+"` WHERE `"+row_element+"` = "+row_element_condition+";"
    #SELECT `storage_tape_status` FROM `Runlog` WHERE `run_number` = 1024;
    if verbose: print(sql)
    try:
        mycursor = connection.cursor()
        mycursor.execute(sql)
        value = mycursor.fetchone()
        if verbose: print(mycursor.rowcount, "Update done")
        mycursor.close()
        return value
    except:
        return -2
    
def update_sql_value(connection, table_name, row_element, row_element_condition, 
                     colum_element, value, verbose=False):
    if isinstance(value, str):
        svalue="\""+value+"\""
    else:
        svalue=str(value)
    if isinstance(row_element_condition, str):
        scondition="\""+row_element_condition+"\""
    else:
        scondition=str(row_element_condition)
    sql = "UPDATE `"+table_name+"` SET `"+colum_element+"` = "+ \
    svalue+" WHERE `"+row_element+"` = "+scondition+";"
    if verbose: print(sql)
    try:
        mycursor = connection.cursor()
        mycursor.execute(sql)
        connection.commit()
        if verbose: print(mycursor.rowcount, "Update done")
        mycursor.close()
        return 0
    except:
        return 1

def daq_sql_cennection(verbose=False):
    import mysql.connector
    import os
    try:
        connection = mysql.connector.connect(
          host=os.environ['MYSQL_IP'],
          user=os.environ['MYSQL_USER'],
          password=os.environ['MYSQL_PASSWORD'],
          database=os.environ['MYSQL_DATABASE'],
          port=int(os.environ['MYSQL_PORT'])
        )
        if verbose: print(connection)
        return connection
    except:
        return False
    
def daq_update_runlog_replica_checksum(connection, run_number, md5sum, verbose=False):
    if verbose: print("md5sum: ", md5sum)
    return update_sql_value(connection, table_name="Runlog", row_element="run_number", 
                     row_element_condition=run_number, 
                     colum_element="file_checksum", value=md5sum, 
                     verbose=verbose)
def daq_update_runlog_replica_tag(connection, run_number, TAG, verbose=False):
    if verbose: print("TAG: ", TAG)
    return update_sql_value(connection, table_name="Runlog", row_element="run_number", 
                     row_element_condition=run_number, 
                     colum_element="file_s3_tag", value=TAG, 
                     verbose=verbose)

def daq_update_runlog_replica_status(connection, run_number, storage, status=-1, verbose=False):
    if storage=="local":
        storage="storage_local_status"
    elif storage=="cloud":
        storage="storage_cloud_status"
    elif storage=="tape":
        storage="storage_tape_status"
    else:
        return 1
    if verbose: print("Storage: "+storage)
    return update_sql_value(connection, table_name="Runlog", row_element="run_number", 
                     row_element_condition=run_number, 
                     colum_element=storage, value=status, 
                     verbose=verbose)
     
    

In [27]:

verbose=True
run_number=268
file_path="/data01/data/run00268.mid.gz"
md5sum=file_md5sum(file_path)
print (md5sum)
connection = daq_sql_cennection(verbose)
daq_update_runlog_replica_status(connection, run_number, storage="local", status=3, verbose=verbose)
daq_update_runlog_replica_checksum(connection, run_number, md5sum, verbose=verbose)
daq_update_runlog_replica_tag(connection, run_number, TAG="LAB", verbose=verbose)

eb5188e7570438d9b47a91565b46b89a
Storage: storage_local_status
UPDATE `Runlog` SET `storage_local_status` = 3 WHERE `run_number` = 268;
0 Update done
md5sum:  eb5188e7570438d9b47a91565b46b89a
UPDATE `Runlog` SET `file_checksum` = "eb5188e7570438d9b47a91565b46b89a" WHERE `run_number` = 268;
0 Update done
TAG:  LAB
UPDATE `Runlog` SET `file_s3_tag` = "LAB" WHERE `run_number` = 268;
0 Update done


0

In [30]:
read_sql_value(connection, table_name="Runlog", row_element="run_number", row_element_condition=str(1024), 
                     colum_element="storage_tape_status", verbose=True)

SELECT `storage_tape_status` FROM `Runlog` WHERE `run_number` = 1024;


-2

In [2]:
def get_content_md5(data):
    import base64
    import hashlib
    digest = hashlib.md5(data).digest()
    return base64.b64encode(digest)

In [3]:
!ls /data01/data/

220321.hst  221107.idf		 run00182.mid.gz      run00513.mid.gz
220321.idf  221107.idx		 run00183.mid.crc32c  run00514.mid.crc32c
220321.idx  221108.hst		 run00183.mid.gz      run00514.mid.gz
220322.hst  221108.idf		 run00184.mid.crc32c  run00515.mid.crc32c
220322.idf  221108.idx		 run00184.mid.gz      run00515.mid.gz
220322.idx  221109.hst		 run00185.mid.crc32c  run00516.mid.crc32c
220323.hst  221109.idf		 run00185.mid.gz      run00516.mid.gz
220323.idf  221109.idx		 run00186.mid.crc32c  run00517.mid.crc32c
220323.idx  221110.hst		 run00186.mid.gz      run00517.mid.gz
220330.hst  221110.idf		 run00187.mid.crc32c  run00518.mid.crc32c
220330.idf  221110.idx		 run00187.mid.gz      run00518.mid.gz
220330.idx  221111.hst		 run00188.mid.crc32c  run00519.mid.crc32c
220331.hst  221111.idf		 run00188.mid.gz      run00519.mid.gz
220331.idf  221111.idx		 run00189.mid.crc32c  run00520.mid.crc32c
220331.idx  221112.hst		 run00189.mid.gz      run00520.mid.gz
220401.hst  221112.idf		 run00190.mid.crc3

In [4]:
get_content_md5("/data01/data/run00759.mid.gz")

TypeError: Unicode-objects must be encoded before hashing

In [6]:
import boto3
boto3.utils.compute_md5("/data01/data/run00759.mid.gz")

AttributeError: module 'boto3.utils' has no attribute 'compute_md5'

In [2]:
def file_as_bytes(file):
    with file:
        return file.read()
file_path="/data01/data/run00759.mid.gz"
def md5sum(file_path):
    import hashlib
    return hashlib.md5(file_as_bytes(open(full_path, 'rb'))).hexdigest()
md5sum(file_path)

'ed0732f7f826b3b6fd99076629489b69'

In [12]:
! md5sum /data01/data/run00759.mid.gz

ed0732f7f826b3b6fd99076629489b69  /data01/data/run00759.mid.gz


In [51]:
0.359/375411430

9.562841493664696e-10

In [52]:
375411430*9.562841493664696e-10

0.35899999999999993

In [55]:
sql = "SELECT * FROM `Runlog` WHERE DATEDIFF(`start_time`, CURRENT_TIMESTAMP) \
BETWEEN 0 AND 30 AND `storage_tape_status` < 1 AND `storage_cloud_status` = 1;"

In [59]:
connection = daq_sql_cennection(verbose)
def daq_not_on_tape_runs(connection):
    sql = "SELECT * FROM `Runlog` WHERE DATEDIFF(`start_time`, CURRENT_TIMESTAMP) \
BETWEEN 0 AND 30 AND `storage_tape_status` < 1 AND `storage_cloud_status` = 1;"
    try:
        mycursor = connection.cursor()
        mycursor.execute(sql)
        value = mycursor.fetchall()
        if verbose: print(mycursor.rowcount)
        mycursor.close()
        return value
    except:
        return -2

In [60]:
daq_not_on_tape_runs(connection)

0


[]